# Operationalize Machine Learning with Amazon SageMaker MLOps

This sequence of notebooks takes you from developing your ML idea in simple notebooks to a production solution with automated model building and CI/CD deployment pipelines.

The labs are organized in the following logical steps:
1. Ingest Data and Build ML model in SageMaker Studio Jupyterlab notebooks
2. Orchestrating an ML pipeline using Visual Editor feature in SageMaker Pipelines. 
3. Add a model building CI/CD pipeline
4. Add a model deployment CI/CD pipeline

![](img/sagemaker-mlops-diagram.jpg)

There are also additional hands-on examples of other SageMaker features and ML topics, like [A/B testing](https://docs.aws.amazon.com/sagemaker/latest/dg/model-validation.html), custom [processing](https://docs.aws.amazon.com/sagemaker/latest/dg/build-your-own-processing-container.html), [training](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-training-algo.html) and [inference](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-inference-main.html) containers, [debugging and profiling](https://docs.aws.amazon.com/sagemaker/latest/dg/train-debugger.html), [security](https://docs.aws.amazon.com/sagemaker/latest/dg/security.html), [multi-model](https://docs.aws.amazon.com/sagemaker/latest/dg/multi-model-endpoints.html) and [multi-container](https://docs.aws.amazon.com/sagemaker/latest/dg/multi-container-endpoints.html) endpoints, and [serial inference pipelines](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-pipelines.html). Explore the notebooks in the folder `additional-topics` to test out these features.

To run this notebook and all notebooks in the workshop please use the `Python 3` kernel in JupyterLab

## Setup
Get the latest version of SageMaker Python SDK.

<div class="alert alert-info"> 💡 The workshop and all notebooks were tested on the SageMaker Python SDK (the package sagemaker) version 2.219.0. The notebooks don't pin the version of the sagemaker. If you encounter any incompatibility issues, you can install the specific version of the sagemaker by running the pip command: <code>%pip install sagemaker=2.219.0</code>
</div>

In [10]:
# Uncomment if you have any compatibility issues and would like to use the specific version of the sagemaker library
# %pip install sagemaker==2.219.0
%pip install --upgrade pip sagemaker boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 92.5 MB/s eta 0:00:00
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.235.0
    Uninstalling sagemaker-2.235.0:
      Successfully uninstalled sagemaker-2.235.0
Note: you may need to restart the kernel to use updated packages.


In [11]:
%pip install mlflow==2.13.2 sagemaker-mlflow

Note: you may need to restart the kernel to use updated packages.


### Import packages

In [12]:
import time
import os
import json
import boto3
import numpy as np  
import pandas as pd 
import sagemaker
from time import gmtime, strftime, sleep

(sagemaker.__version__,boto3.__version__)

('2.235.0', '1.35.65')

In [13]:
import time
import os
import json
import boto3
import numpy as np  
import pandas as pd 
import sagemaker
from time import gmtime, strftime, sleep

(sagemaker.__version__,boto3.__version__)

('2.235.0', '1.35.65')

In [14]:
import json
def write_params(step_name, params):
    with open(f"{step_name}.json", "w") as f:    
        f.write(json.dumps(params))

def read_params(step_name):
    with open(f"{step_name}.json", "r") as f:
        data = f.read()
        params = json.loads(data)
    return params

### Instantiate variables

In [15]:
# Get some variables you need to interact with SageMaker service
bucket_name = sagemaker.Session().default_bucket()
sm_role = sagemaker.get_execution_role()
print(sm_role)

arn:aws:iam::141310854322:role/cfnstudiodomain-SageMakerExecutionRole-ZVpJq8dZPMu8


### Connect to MLflow tracking server
If you're running an AWS-led workshop or used the delivered CloudFormation template to provision your workshop environment, an MLflow server must be up and running. If you don't have an MLflow server, follow the [Developer Guide](https://docs.aws.amazon.com/sagemaker/latest/dg/mlflow-create-tracking-server.html) or run the following code cell to create a new one.

To create and manage an MLflow tracking server and to work with managed MLflow experiements, you need the following permissions attached to the SageMaker execution role:

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "sagemaker-mlflow:*",
                "sagemaker:CreateMlflowTrackingServer",
                "sagemaker:UpdateMlflowTrackingServer",
                "sagemaker:DeleteMlflowTrackingServer",
                "sagemaker:StartMlflowTrackingServer",
                "sagemaker:StopMlflowTrackingServer",
                "sagemaker:CreatePresignedMlflowTrackingServerUrl"
            ],
            "Resource": "*"
        }
    ]
}
```

Execute the following code to check if you have a running MLflow server.

In [16]:
# Find an active MLflow server in the account
r = boto3.client("sagemaker").list_mlflow_tracking_servers(
    TrackingServerStatus='Created',
)['TrackingServerSummaries']

if len(r) < 1:
    print("You don't have any running MLflow servers. Trying to find a server in the status 'Creating'...")

    r = boto3.client("sagemaker").list_mlflow_tracking_servers(
        TrackingServerStatus='Creating',
    )['TrackingServerSummaries']

    if len(r) < 1:
        print("You don't have any MLflow server in the status 'Creating'. Run the next code cell to create a new one.")
        mlflow_arn = None
        mlflow_name = None
    else:
        mlflow_arn = r[0]['TrackingServerArn']
        mlflow_name = r[0]['TrackingServerName']
        print(f"You have an MLflow server {mlflow_arn} in the status 'Creating', going to use this one")
else:
    mlflow_arn = r[0]['TrackingServerArn']
    mlflow_name = r[0]['TrackingServerName']
    print(f"You have {len(r)} running MLflow server(s). Get the first server ARN:{mlflow_arn}")

You have 1 running MLflow server(s). Get the first server ARN:arn:aws:sagemaker:us-east-1:141310854322:mlflow-tracking-server/mlflow-d-ms17jq3aatnw


In [17]:
# This code cell creates a new MLflow server
if not mlflow_arn:
    ts = strftime('%d-%H-%M-%S', gmtime())
    mlflow_name = f"mlflow-{ts}"
    r = boto3.client("sagemaker").create_mlflow_tracking_server(
        TrackingServerName=mlflow_name,
        ArtifactStoreUri=f"s3://{bucket_name}/mlflow/{ts}",
        RoleArn=sm_role,
        AutomaticModelRegistration=True,
    )

    mlflow_arn = r['TrackingServerArn']
    print(f"Server creation request succeded. The server {mlflow_arn} is being created.")

<div style="border: 4px solid coral; text-align: center; margin: auto;">
Creation of an MLflow server can take up to 25 minutes. You don't need to wait - proceed with the flow of the workshop.
</div>

In [18]:
(mlflow_arn, mlflow_name)

('arn:aws:sagemaker:us-east-1:141310854322:mlflow-tracking-server/mlflow-d-ms17jq3aatnw',
 'mlflow-d-ms17jq3aatnw')

# Summary and Next Step
In this notebook, we performed the steps to validate the required infrastructures needed to run the workshop successfully. For instance, setting up an MLFlow tracking server. Once the notebook is complete, you can continue the workshop instruction to execute the subsequent labs to build the MLOps on SageMaker.